[![Image Name](https://cdn.kesci.com/upload/sehv2zq3qx.jpg?imageView2/0/w/960/h/960)](https://www.heywhale.com/home/competition/66598b3271a1fd975a17d6ad)  
[**vgbhfive**](http://blog.vgbhfive.com)，多年风控引擎研发及金融模型开发经验，现任某公司风控研发工程师，对数据分析、金融模型开发、风控引擎研发具有丰富经验。

前一关卡中我们学习了 `SVM` 支持向量机模型的搭建，其中的核函数可以说是其中的重点，其通过扩展维度空间的方式，使得不可分的数据变成可分的原因就是核函数。下面我们开始学习比赛中的大杀器 - `XGBoost`。

### XGBoost  
说起 `XGBoost` 就不得不说 `GBDT` 算法，`XGBoost` 是 `GBDT` 算法的一种实现，其在算法和工程上进行了很多的改进，被广泛的应用于很多的比赛中，堪称“比赛大杀器”。  

`XGBoost` 对应的模型是**复合树模型**，复合树模型是一组分类和回归树。通常情况下，在实践中往往一棵树是不够用的，此时需要把**多棵树的预测结果**总和起来，这就是复合树模型。

### 葡萄酒数据集之多分类问题  
在红酒行业中，品鉴师由于其专业度受到很多行业人士的信赖，其最重要的一项工作就是品鉴红酒，但鉴于品酒师的人数发展，现需要开发一款用于分类红酒品种的机器学习模型，通过分类不同红酒中各种元素含量来区分不同的红酒品种。该数据集包含178 例记录，其中收集了红酒的酒精度数、苹果酸含量、颜色、颜色强度、黄胺类含量等一系列指标。

葡萄酒数据集中特征含义如下：  

| 特征列名称	| 特征含义|  
| ---- | ---- |  
| alcohol   | 酒精  |  
| malic_acid   | 苹果酸  |  
| ash  | 灰  |  
|alcalinity_of_ash   | 灰分的碱度  |  
| magnesium  | 镁  |  
| total_phenols  | 总酚  |  
| flavanoids  | 黄酮类化合物  |  
| nonflavanoid_phenols  | 非黄烷类酚类  |  
| proanthocyanins  | 原花色素  |  
| color_intensity   | 颜色强度  |  
| hue  | 色调  |  
| od280/od315_of_diluted_wines  | 稀释葡萄酒的OD280/OD315  |  
| proline  | 脯氨酸  |  

在结果分类中第一类有 `59` 个样本，第二类有 `71` 个样本，第三类有 `48` 个样本。

#### 引入依赖

In [74]:
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.datasets import load_wine, load_diabetes
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from xgboost import XGBClassifier, XGBRegressor
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestClassifier

#### 加载数据

In [75]:
# 1. 加载数据

wine = load_wine()
x = pd.DataFrame(wine.data, columns=wine.feature_names)
y = pd.DataFrame(wine.target)
x.head(), y.head()

(   alcohol  malic_acid   ash  alcalinity_of_ash  magnesium  total_phenols  \
 0    14.23        1.71  2.43               15.6      127.0           2.80   
 1    13.20        1.78  2.14               11.2      100.0           2.65   
 2    13.16        2.36  2.67               18.6      101.0           2.80   
 3    14.37        1.95  2.50               16.8      113.0           3.85   
 4    13.24        2.59  2.87               21.0      118.0           2.80   
 
    flavanoids  nonflavanoid_phenols  proanthocyanins  color_intensity   hue  \
 0        3.06                  0.28             2.29             5.64  1.04   
 1        2.76                  0.26             1.28             4.38  1.05   
 2        3.24                  0.30             2.81             5.68  1.03   
 3        3.49                  0.24             2.18             7.80  0.86   
 4        2.69                  0.39             1.82             4.32  1.04   
 
    od280/od315_of_diluted_wines  proline  
 0  

#### 数据基础性分析

In [76]:
# 2. 数据基础性分析

x.info()

# 数据集中不存在空值，由于 XGBoost 模型本身具有处理缺失值的能力，一般来说直接使用 np.nan 作为缺失值处理是最好的。

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178 entries, 0 to 177
Data columns (total 13 columns):
alcohol                         178 non-null float64
malic_acid                      178 non-null float64
ash                             178 non-null float64
alcalinity_of_ash               178 non-null float64
magnesium                       178 non-null float64
total_phenols                   178 non-null float64
flavanoids                      178 non-null float64
nonflavanoid_phenols            178 non-null float64
proanthocyanins                 178 non-null float64
color_intensity                 178 non-null float64
hue                             178 non-null float64
od280/od315_of_diluted_wines    178 non-null float64
proline                         178 non-null float64
dtypes: float64(13)
memory usage: 18.2 KB


In [77]:
y[0].value_counts()

1    71
0    59
2    48
Name: 0, dtype: int64

#### 计算特征重要性

In [78]:
# 3. 使用随机森林计算特征重要性

forest = RandomForestClassifier()
forest.fit(x, y)
importance = pd.DataFrame({"columns": wine.feature_names, "importances": forest.feature_importances_})
importance.sort_values(by='importances', ascending=False)

# 在计算得出特征重要性排名之后，可以根据排名酌情删除重要性太低的特征，避免其影响模型拟合。

/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


,columns,importances
9,color_intensity,0.213239
10,hue,0.147334
12,proline,0.128770
6,flavanoids,0.116462
11,od280/od315_of_diluted_wines,0.087261
1,malic_acid,0.062757
3,alcalinity_of_ash,0.062711
5,total_phenols,0.061623
0,alcohol,0.048133
4,magnesium,0.035127


In [79]:
# 4. 根据特征重要性（确定重要性的阈值，这里删除的是重要性小于 0.02 的特征），删除个别特征
x = x.drop(['ash', 'nonflavanoid_phenols', 'proanthocyanins'], axis=1)
x.head()

,alcohol,malic_acid,alcalinity_of_ash,magnesium,total_phenols,flavanoids,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,14.23,1.71,15.6,127.0,2.80,3.06,5.64,1.04,3.92,1065.0
1,13.20,1.78,11.2,100.0,2.65,2.76,4.38,1.05,3.40,1050.0
2,13.16,2.36,18.6,101.0,2.80,3.24,5.68,1.03,3.17,1185.0
3,14.37,1.95,16.8,113.0,3.85,3.49,7.80,0.86,3.45,1480.0
4,13.24,2.59,21.0,118.0,2.80,2.69,4.32,1.04,2.93,735.0


#### 训练模型

In [80]:
# 5. 划分训练集和测试集

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)
x_train.head(), x_test.head(), y_train.head(), y_test.head()

(     alcohol  malic_acid  alcalinity_of_ash  magnesium  total_phenols  \
 138    13.49        3.59               19.5       88.0           1.62   
 104    12.51        1.73               20.5       85.0           2.20   
 78     12.33        0.99               14.8      136.0           1.90   
 36     13.28        1.64               15.5      110.0           2.60   
 93     12.29        2.83               18.0       88.0           2.45   
 
      flavanoids  color_intensity   hue  od280/od315_of_diluted_wines  proline  
 138        0.48             5.70  0.81                          1.82    580.0  
 104        1.92             2.94  1.04                          3.57    672.0  
 78         1.85             3.40  1.06                          2.31    750.0  
 36         2.68             4.60  1.09                          2.78    880.0  
 93         2.25             2.15  1.15                          3.30    290.0  ,
      alcohol  malic_acid  alcalinity_of_ash  magnesium  total_phen

In [44]:
# 6. 构建xgb 模型

model = XGBClassifier(max_depth=3, learning_rate=0.5, n_estimators=50, gamma=0.5, min_child_weight=5, random_state=42)
model.fit(x_train, y_train)

/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bytree=1, gamma=0.5, learning_rate=0.5,
              max_delta_step=0, max_depth=3, min_child_weight=5, missing=None,
              n_estimators=50, n_jobs=1, nthread=None,
              objective='multi:softprob', random_state=42, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
              subsample=1)

#### 预测测试集并计算指标

In [45]:
# 7. 预测测试集

y_pred = model.predict(x_test)

acc = accuracy_score(y_test, y_pred)
acc

1.0

#### 模型调参

In [10]:
# 8. 模型调参

model = XGBClassifier(random_state=42)
param_grid = {
    "max_depth": [3, 5, 7, 10],
    "learning_rate": [0.01, 0.1, 0.5],
    "n_estimators": [50, 100, 200],
    "gamma": [0, 0.1, 0.2, 0.5],
    "min_child_weight": [1, 3, 5]
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy', verbose=2)
grid_search.fit(x_train, y_train)

Fitting 5 folds for each of 432 candidates, totalling 2160 fits
[CV] gamma=0, learning_rate=0.01, max_depth=3, min_child_weight=1, n_estimators=50 
[CV]  gamma=0, learning_rate=0.01, max_depth=3, min_child_weight=1, n_estimators=50, total=   0.0s
[CV] gamma=0, learning_rate=0.01, max_depth=3, min_child_weight=1, n_estimators=50 
[CV]  gamma=0, learning_rate=0.01, max_depth=3, min_child_weight=1, n_estimators=50, total=   0.0s
[CV] gamma=0, learning_rate=0.01, max_depth=3, min_child_weight=1, n_estimators=50 
[CV]  gamma=0, learning_rate=0.01, max_depth=3, min_child_weight=1, n_estimators=50, total=   0.0s
[CV] gamma=0, learning_rate=0.01, max_depth=3, min_child_weight=1, n_estimators=50 
[CV]  gamma=0, learning_rate=0.01, max_depth=3, min_child_weight=1, n_estimators=50, total=   0.0s
[CV] gamma=0, learning_rate=0.01, max_depth=3, min_child_weight=1, n_estimators=50 
[CV]  gamma=0, learning_rate=0.01, max_depth=3, min_child_weight=1, n_estimators=50, total=   0.0s
[CV] gamma=0, learnin

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.6/site-packages/sklearn/

[CV]  gamma=0, learning_rate=0.01, max_depth=3, min_child_weight=1, n_estimators=100, total=   0.0s
[CV] gamma=0, learning_rate=0.01, max_depth=3, min_child_weight=1, n_estimators=100 
[CV]  gamma=0, learning_rate=0.01, max_depth=3, min_child_weight=1, n_estimators=100, total=   0.0s
[CV] gamma=0, learning_rate=0.01, max_depth=3, min_child_weight=1, n_estimators=100 
[CV]  gamma=0, learning_rate=0.01, max_depth=3, min_child_weight=1, n_estimators=100, total=   0.0s
[CV] gamma=0, learning_rate=0.01, max_depth=3, min_child_weight=1, n_estimators=100 
[CV]  gamma=0, learning_rate=0.01, max_depth=3, min_child_weight=1, n_estimators=100, total=   0.0s
[CV] gamma=0, learning_rate=0.01, max_depth=3, min_child_weight=1, n_estimators=200 
[CV]  gamma=0, learning_rate=0.01, max_depth=3, min_child_weight=1, n_estimators=200, total=   0.1s
[CV] gamma=0, learning_rate=0.01, max_depth=3, min_child_weight=1, n_estimators=200 
[CV]  gamma=0, learning_rate=0.01, max_depth=3, min_child_weight=1, n_estim

/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for 

[CV]  gamma=0, learning_rate=0.01, max_depth=3, min_child_weight=1, n_estimators=200, total=   0.1s
[CV] gamma=0, learning_rate=0.01, max_depth=3, min_child_weight=1, n_estimators=200 


...

[CV]  gamma=0.5, learning_rate=0.5, max_depth=10, min_child_weight=3, n_estimators=200, total=   0.0s
[CV] gamma=0.5, learning_rate=0.5, max_depth=10, min_child_weight=3, n_estimators=200 
[CV]  gamma=0.5, learning_rate=0.5, max_depth=10, min_child_weight=3, n_estimators=200, total=   0.0s
[CV] gamma=0.5, learning_rate=0.5, max_depth=10, min_child_weight=3, n_estimators=200 
[CV]  gamma=0.5, learning_rate=0.5, max_depth=10, min_child_weight=3, n_estimators=200, total=   0.0s
[CV] gamma=0.5, learning_rate=0.5, max_depth=10, min_child_weight=3, n_estimators=200 
[CV]  gamma=0.5, learning_rate=0.5, max_depth=10, min_child_weight=3, n_estimators=200, total=   0.0s
[CV] gamma=0.5, learning_rate=0.5, max_depth=10, min_child_weight=5, n_estimators=50 
[CV]  gamma=0.5, learning_rate=0.5, max_depth=10, min_child_weight=5, n_estimators=50, total=   0.0s
[CV] gamma=0.5, learning_rate=0.5, max_depth=10, min_child_weight=5, n_estimators=50 
[CV]  gamma=0.5, learning_rate=0.5, max_depth=10, min_chil

/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for 

[CV]  gamma=0.5, learning_rate=0.5, max_depth=10, min_child_weight=5, n_estimators=100, total=   0.0s
[CV] gamma=0.5, learning_rate=0.5, max_depth=10, min_child_weight=5, n_estimators=100 
[CV]  gamma=0.5, learning_rate=0.5, max_depth=10, min_child_weight=5, n_estimators=100, total=   0.0s
[CV] gamma=0.5, learning_rate=0.5, max_depth=10, min_child_weight=5, n_estimators=100 
[CV]  gamma=0.5, learning_rate=0.5, max_depth=10, min_child_weight=5, n_estimators=100, total=   0.0s
[CV] gamma=0.5, learning_rate=0.5, max_depth=10, min_child_weight=5, n_estimators=100 
[CV]  gamma=0.5, learning_rate=0.5, max_depth=10, min_child_weight=5, n_estimators=100, total=   0.0s
[CV] gamma=0.5, learning_rate=0.5, max_depth=10, min_child_weight=5, n_estimators=200 
[CV]  gamma=0.5, learning_rate=0.5, max_depth=10, min_child_weight=5, n_estimators=200, total=   0.0s
[CV] gamma=0.5, learning_rate=0.5, max_depth=10, min_child_weight=5, n_estimators=200 
[CV]  gamma=0.5, learning_rate=0.5, max_depth=10, min_c

/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for 

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bytree=1,
                                     gamma=0, learning_rate=0.1,
                                     max_delta_step=0, max_depth=3,
                                     min_child_weight=1, missing=None,
                                     n_estimators=100, n_jobs=1, nthread=None,
                                     objective='binary:logistic',
                                     random_state=42, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=None, silent=True,
                                     subsample=1),
             iid='warn', n_jobs=None,
             param_grid={'gamma': [0, 0.1, 0.2, 0.5],
                         'learning_rate': [0.01, 0.1, 0.5],
                         'max_depth': [3, 5, 7, 10],
                         'min_ch

In [11]:
# 9. 最佳参数和最高准确率

grid_search.best_params_, grid_search.best_score_

# 在拿到最佳参数之后就可以返回调整模型参数从而训练出准确率更高的模型

({'gamma': 0.5,
  'learning_rate': 0.5,
  'max_depth': 3,
  'min_child_weight': 5,
  'n_estimators': 50},
 0.967741935483871)

In [96]:
model = XGBClassifier(max_depth=10, learning_rate=0.5, n_estimators=50, gamma=0.5, min_child_weight=5, random_state=42)
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
mse = mean_squared_error(y_test, y_pred)
print(f"{mse:.2f}")  

0.00


/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


### 糖尿病数据集之预测指标

糖尿病是一种慢性代谢性疾病，长期高血糖可导致多种并发症，如心血管疾病、肾病、视网膜病变等。早期诊断和及时治疗对于控制糖尿病及其并发症具有重要意义，但在传统诊断方法依赖于血液检测，这不仅成本较高，而且需要患者定期到医院进行检测。该数据集有442 例记录，其中收集了一定数量的糖尿病患者和健康人群的医学数据，包括年龄、性别、体重指数(BMI)、血压、血糖水平以及一年内是否患糖尿病等特征信息。

糖尿病数据集特征含义如下：  

|特征列名称 |	特征含义|  
| ---- | ---- |  
| 0   | 年龄  |  
| 1   | 性别  |  
| 2   | BMI  |  
| 3   | BP  |  
| 4  | s1  |  
| 5  | s2  |  
| 6  | s3  |  
| 7  | s4  |  
| 8  | s5  |

#### 加载数据

In [57]:
# 1. 加载数据

diabetes = load_diabetes()
x = pd.DataFrame(diabetes.data)
y = diabetes.target
x.head(), y

(          0         1         2         3         4         5         6  \
 0  0.038076  0.050680  0.061696  0.021872 -0.044223 -0.034821 -0.043401   
 1 -0.001882 -0.044642 -0.051474 -0.026328 -0.008449 -0.019163  0.074412   
 2  0.085299  0.050680  0.044451 -0.005671 -0.045599 -0.034194 -0.032356   
 3 -0.089063 -0.044642 -0.011595 -0.036656  0.012191  0.024991 -0.036038   
 4  0.005383 -0.044642 -0.036385  0.021872  0.003935  0.015596  0.008142   
 
           7         8         9  
 0 -0.002592  0.019908 -0.017646  
 1 -0.039493 -0.068330 -0.092204  
 2 -0.002592  0.002864 -0.025930  
 3  0.034309  0.022692 -0.009362  
 4 -0.002592 -0.031991 -0.046641  ,
 array([151.,  75., 141., 206., 135.,  97., 138.,  63., 110., 310., 101.,
         69., 179., 185., 118., 171., 166., 144.,  97., 168.,  68.,  49.,
         68., 245., 184., 202., 137.,  85., 131., 283., 129.,  59., 341.,
         87.,  65., 102., 265., 276., 252.,  90., 100.,  55.,  61.,  92.,
        259.,  53., 190., 142.,  75

#### 数据预处理

In [58]:
# 2. 数据预处理

# 初始化特征缩放器和缺失值填充器
scaler = StandardScaler()
impute = SimpleImputer(strategy='mean')

# 缩放特征和填充缺失值
x_scaled = scaler.fit_transform(x)
y = impute.fit_transform(y.reshape(-1, 1)).flatten()

x_scaled, y

(array([[ 0.80050009,  1.06548848,  1.29708846, ..., -0.05449919,
          0.41855058, -0.37098854],
        [-0.03956713, -0.93853666, -1.08218016, ..., -0.83030083,
         -1.43655059, -1.93847913],
        [ 1.79330681,  1.06548848,  0.93453324, ..., -0.05449919,
          0.06020733, -0.54515416],
        ...,
        [ 0.87686984,  1.06548848, -0.33441002, ..., -0.23293356,
         -0.98558469,  0.32567395],
        [-0.9560041 , -0.93853666,  0.82123474, ...,  0.55838411,
          0.93615545, -0.54515416],
        [-0.9560041 , -0.93853666, -1.53537419, ..., -0.83030083,
         -0.08871747,  0.06442552]]),
 array([151.,  75., 141., 206., 135.,  97., 138.,  63., 110., 310., 101.,
         69., 179., 185., 118., 171., 166., 144.,  97., 168.,  68.,  49.,
         68., 245., 184., 202., 137.,  85., 131., 283., 129.,  59., 341.,
         87.,  65., 102., 265., 276., 252.,  90., 100.,  55.,  61.,  92.,
        259.,  53., 190., 142.,  75., 142., 155., 225.,  59., 104., 182.,
   

#### 划分训练集和测试集

In [59]:
# 3. 划分训练集和测试集

x_train, x_test, y_train, y_test = train_test_split(x_scaled, y, test_size=0.3, random_state=42)
x_train, x_test, y_train, y_test

(array([[ 0.6477606 ,  1.06548848,  0.68527653, ...,  0.72130245,
          1.32484846,  0.06442552],
        [ 1.56419757, -0.93853666,  1.79560189, ..., -0.05449919,
          0.13054339,  1.80608174],
        [-1.18511334,  1.06548848, -0.22111152, ...,  0.72130245,
          0.43334944,  1.19650206],
        ...,
        [ 0.6477606 , -0.93853666, -0.42504883, ..., -0.83030083,
         -0.22925264, -0.02265729],
        [-0.26867637, -0.93853666, -0.49302793, ..., -0.05449919,
         -0.80855653, -0.80640259],
        [-1.94881082, -0.93853666,  0.59463772, ..., -0.83030083,
         -0.10817391, -0.02265729]]),
 array([[ 0.95323959, -0.93853666, -0.13047271, ...,  0.72130245,
          0.68186952, -0.1097401 ],
        [ 1.9460463 , -0.93853666,  0.77591533, ..., -0.83030083,
         -0.47329131, -0.45807135],
        [ 1.33508832,  1.06548848, -0.08515331, ..., -0.05449919,
          1.77641158, -0.37098854],
        ...,
        [ 1.33508832,  1.06548848, -0.03983391, ...,  

##### 特征重要性

In [60]:
# 4. 预训练模型查看特征重要性，之后根据特征重要性选择重要特征

tmp_model = XGBRegressor(objective='reg:linear', random_state=42)
tmp_model.fit(x_train, y_train)
feature_importance = pd.DataFrame({'Feature': diabetes.feature_names, 'Importance': tmp_model.feature_importances_})
print(feature_importance)

selected_features = feature_importance[feature_importance['Importance'] > 0.05].index.tolist()
selected_features

  Feature  Importance
0     age    0.043864
1     sex    0.056586
2     bmi    0.327141
3      bp    0.087945
4      s1    0.063625
5      s2    0.045623
6      s3    0.078824
7      s4    0.098734
8      s5    0.139373
9      s6    0.058286


[1, 2, 3, 4, 6, 7, 8, 9]

In [61]:
# 5. 使用重要特征训练模型

x_train = x_train[:, selected_features]
x_test = x_test[:, selected_features]
x_train, x_test

(array([[ 1.06548848,  0.68527653,  1.03889762, ...,  0.72130245,
          1.32484846,  0.06442552],
        [-0.93853666,  1.79560189,  1.32842647, ..., -0.05449919,
          0.13054339,  1.80608174],
        [ 1.06548848, -0.22111152,  0.53222214, ...,  0.72130245,
          0.43334944,  1.19650206],
        ...,
        [-0.93853666, -0.42504883, -0.11921776, ..., -0.83030083,
         -0.22925264, -0.02265729],
        [-0.93853666, -0.49302793, -0.84303987, ..., -0.05449919,
         -0.80855653, -0.80640259],
        [-0.93853666,  0.59463772, -0.33636439, ..., -0.83030083,
         -0.10817391, -0.02265729]]),
 array([[-0.93853666, -0.13047271, -0.33636439, ...,  0.72130245,
          0.68186952, -0.1097401 ],
        [-0.93853666,  0.77591533,  0.45983993, ..., -0.83030083,
         -0.47329131, -0.45807135],
        [ 1.06548848, -0.08515331, -0.26398218, ..., -0.05449919,
          1.77641158, -0.37098854],
        ...,
        [ 1.06548848, -0.03983391,  1.40080868, ...,  

#### 训练模型

In [64]:
# 6. 使用 XGBRegressor 构建模型

regressor = XGBRegressor(objective='reg:linear', n_estimators=100, learning_rate=0.05, random_state=42)
regressor.fit(x_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bytree=1, gamma=0, importance_type='gain',
             learning_rate=0.05, max_delta_step=0, max_depth=3,
             min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
             nthread=None, objective='reg:linear', random_state=42, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
             subsample=1)

`XGBRegressor(objective, **kwargs)` 重要参数含义如下：  
 - `objective`: 目标函数，可用列表如下：  
    - 回归任务： `reg:liner` （默认）、`reg:logistic`。  
    - 二分类：`binary:logistic` 概率、`binary:logitraw` 类别。  
    - 多分类：`multi:softmax` `num_class=n` 返回类别、 `multi:softprob` `num_class=n` 返回类别、`rank:pairwise`。  
 - `n_estimators`: 梯度提升树的数量，相当于推导次数。  
 - `max_depth`: 树的最大深度。  
 - `learning_rate`: 学习率。  
 - `max_leaves`: 最大叶数量，`0` 表示没限制。  
 - `n_jobs`: 并行运行的线程数。  
 - `min_child_weight`: 子树最小权重总和。  
 - `subsample`: 训练实例的子样本比率。

#### 预测测试集并计算指标

In [65]:
# 7. 预测测试集

y_pred = regressor.predict(x_test)

# R2 决定系数（拟合优度），用于评估回归任务，其值越大越好
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mse, r2

(2781.644805884221, 0.4847191141039273)

#### 回归交叉验证

In [66]:
# 回归交叉验证

kf = KFold(n_splits=5, shuffle=True, random_state=42)
for train_index, test_index in kf.split(x_scaled):
    xgb_model = XGBRegressor(objective='reg:linear', 
                             n_estimators=100,               
                             learning_rate=0.05,
                             random_state=42).fit(x_scaled[train_index], y[train_index])
    
    pred = xgb_model.predict(x_scaled[test_index])
    print('-'*50)
    print('mse', mean_squared_error(y[test_index], pred))
    print('r2_score', r2_score(y[test_index], pred))
    print('score', xgb_model.score(x_scaled[test_index], y[test_index]))

--------------------------------------------------
mse 2780.9313712037906
r2_score 0.47511292556269136
score 0.4751129255626914
--------------------------------------------------
mse 2906.7458182737014
r2_score 0.5340715251658241
score 0.5340715251658241
--------------------------------------------------
mse 3941.2430881562755
r2_score 0.27589660178767106
score 0.27589660178767106
--------------------------------------------------
mse 3358.0721931533308
r2_score 0.5010557381303739
score 0.5010557381303739
--------------------------------------------------
mse 3241.320024127687
r2_score 0.41990634791156134
score 0.41990634791156134


### 总结  
在模型的训练过程中，最重要的内容便是特征的基础性分析，在足够了解数据的基础上，通过特征归一化、计算特征重要性、特征分段分析、逐个计算特征与结果的相关性等操作，从全局出发通过模型参数优化、交叉验证等手段，使得模型达到其巅峰状态。

### 闯关题

#### STEP1：请根据要求完成题目

Q1. 模型优化中的预填参数如何确定？  
  A. 随便写，反正最终都会给出最优答案。  
  B. 结合数据集和经验填写。  
  C. 不写。

Q2. 关于 k 折交叉验证，下列说法正确的是？  
  A. k 值并不是越大越好，k 值过大，会降低运算速度  
  B. 选择更大的 k 值，会让偏差更小，因为 k 值越大，训练集越接近整个训练样本  
  C. 选择合适的 k 值，能减小验方差  
  D. 以上说法都正确

Q3. 葡萄酒数据集中模型的mse 均方误差是多少？（小数点后两位四舍五入）  
   A. 0.01  
   B. 0.02  
   C. 0.03

Q4. 葡萄酒模型预测某个测试集的答案是什么？  
   (13.56, 1.73, 2.46, 20.5, 116.0, 2.96, 2.78, 0.2, 2.45, 6.25, 0.98, 3.03, 1120.0)  
   A. 0  
   B. 1  
   C. 2

In [69]:
df_tmp = pd.DataFrame({"alcohol": 13.56, "malic_acid": 1.73, "alcalinity_of_ash": 20.5, "magnesium": 116.0, 
    "total_phenols": 2.96, "flavanoids": 2.78,"color_intensity": 6.25, "hue": 0.98, "od280/od315_of_diluted_wines": 3.03, "proline": 1120.0}, index=[0])
df_tmp
pre = model.predict(df_tmp)
# 预测数据
print(pre[0])

0


In [105]:
#填入你的答案并运行,注意大小写
a1 = 'B'  # 如 a1= 'A'
a2 = 'D'  # 如 a2= 'A'
a3 = 'B'  # 如 a3= 'A'
a4 = 'A'  # 如 a4= 'A'

#### STEP2：将结果保存为 csv 文件  
将结果保存为 csv 文件  
csv 需要有两列，列名：id、answer。其中，id 列为题号，如 q1、q2；answer 列为 STEP1 中各题你计算出来的结果。💡 这一步的代码你不用做任何修改，直接运行即可。

In [106]:
import pandas as pd

# 生成 csv 作业答案文件
def save_csv(a1, a2, a3, a4):
    df = pd.DataFrame({"id": ["q1", "q2", "q3", "q4"], "answer": [a1, a2, a3, a4]})
    df.to_csv("answer_5.csv", index=None)
    print(df)

save_csv(a1, a2, a3, a4)

   id answer
0  q1      B
1  q2      D
2  q3      B
3  q4      A


#### STEP3: 提交 csv 文件，获取分数结果  
提交 csv 文件，获取分数结果  

你的 csv 答案文件已经准备完毕了，最后让我们提交答案文件，看看是否正确。  

提交方法：  

1、拷贝提交 token  

去对应关卡的 提交页面，找到对应关卡，看到了你的 token 嘛？  

拷贝它。  

记得：每个关卡的 token 不一样。  

2、下方 cell 里，拿你拷贝的 token 替换掉 XXXXXXX， 然后 Ctrl+Enter 运行 。  



In [107]:
#运行这个Cell 下载提交工具

!wget -nv -O heywhale_submit https://cdn.kesci.com/submit_tool/v4/heywhale_submit&&chmod +x heywhale_submit

# 运行提交工具
# 把下方XXXXXXX替换为你的 Token
# 改完看起来像是：!./heywhale_submit -token 586eeef71cb92941 -file answer_5.csv

!./heywhale_submit -token de2a94dd514b832c -file answer_5.csv  # 替换XXXXXXX；注意不可增减任何空格或其他字符

wget: /opt/conda/lib/libcrypto.so.1.0.0: no version information available (required by wget)
wget: /opt/conda/lib/libssl.so.1.0.0: no version information available (required by wget)
wget: /opt/conda/lib/libssl.so.1.0.0: no version information available (required by wget)
2025-07-18 06:01:49 URL:https://cdn.kesci.com/submit_tool/v4/heywhale_submit [22020102/22020102] -> "heywhale_submit" [1]
Heywhale Submit Tool: v5.0.1 
> 已验证Token
> 开始上传文件
30 / 30 [||||||||||||||||||||||||||||||||||||||||||||||||||||||||] ? p/s 100.00%
> 文件已上传        
> 服务器响应: 200 提交成功，请等待评审完成
> 提交完成
